<a href="https://colab.research.google.com/github/johyunkang/MLwithPythonCookbook/blob/main/10_%ED%8A%B9%EC%84%B1%EC%84%A0%ED%83%9D%EC%9D%84%EC%82%AC%EC%9A%A9%ED%95%9C_%EC%B0%A8%EC%9B%90%EC%B6%95%EC%86%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 10.0 소개

9장에서 새로운 특성을 만드는 식으로 특성 행렬의 차원을 축소하는 방법을 설명했음

이런 특성은 (이상적으로) 훨씬 적은 차원으로 좋은 품질의 모델을 동일하게 훈련할 수 있음.

이를 `특성 추출(feature extraction)`이라고 부름

이 장에서 또 다른 접근 방법으로 고품질의 정보가 많은 특성은 선택하고 덜유용한 특성은 버리는 방식을 다루겠음

이를 `특성 선택(feature selection)`이라고 부름

**특성 선택** 방식에는 필터(`filter`), 래퍼(`wrapper`), 임베디드(`embeded`) 세 가지가 있음
- 필터 방식은 통계적인 속성을 조사하여 가장 뛰어난 특성을 선택
- 래퍼 방식은 시행착오를 통해 가장 높은 품질의 예측을 만드는 특성의 부분 조합을 찾음
- 임베디드 방식은 학습 알고리즘의 훈련 단계를 확장하거나 일부로 구성하여 가장 좋은 특성의 부분 조합을 선택

## 10.1 분산을 기준으로 수치 특성 선택하기

수치형 특성 중에서 분산이 낮은 특성 (즉, 정보가 거의 없는 특성)을 삭제하고 싶음

In [4]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris()

feature = iris.data
target = iris.target

print('feature shape:', feature.shape, ', target shape:', target.shape)
print('feature sample:', feature[:3])
print('target sample:', target[:3])

feature shape: (150, 4) , target shape: (150,)
feature sample: [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]
target sample: [0 0 0]


In [5]:
# 기준값 만들기
threshold = VarianceThreshold(threshold=0.5)

# 기준값 보다 높은 특성을 선택
features_high_variance = threshold.fit_transform(feature)

# 선택한 특성 확인
print('선택한 특성 shape:', features_high_variance.shape)
print('선택한 특성 확인:', features_high_variance[:3])

선택한 특성 shape: (150, 3)
선택한 특성 확인: [[5.1 1.4 0.2]
 [4.9 1.4 0.2]
 [4.7 1.3 0.2]]


`분산기준설정(variance thresholding; VT)`를 사용할 때는 두 가지를 기억해야 함
- 분산은 원점에 맞춰진 값이 아님. 즉 특성의 제곱 단위임. 따라서 특성의 단위가 서로 다르면 VT가 작동하지 않음
- 분산의 기준값을 수동으로 선택하기 때문에 어떤 값이 좋은지 판단할 수 있어야 함.

In [7]:
print('분산을 확인합니다:', threshold.variances_)

분산을 확인합니다: [0.68112222 0.18871289 3.09550267 0.57713289]


- 특성이 (평균이 0이고 단위 분산으로) 표준화되어 있으면 당연하지만 분산 기준 선택 방식은 올바르게 작동하지 않습니다.

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
feature_std = scaler.fit_transform(feature)

# 각 특성의 분산을 계산
selector = VarianceThreshold()
print('표준화된 분산 확인 시...:', selector.fit(feature_std).variances_)

표준화된 분산 확인 시...: [1. 1. 1. 1.]


## 10.2 분산을 기준으로 이진 특성 선택하기 

**이진 범주형 특성(binary categorical feature)**에서 분산이 낮은 특성(적은 정보를 가진 특성)을 삭제하고 싶음

`베르누이 확률 변수(Bernoulli random variable)`의 분산이 기준값 이상인 특성을 선택

In [ ]:
from sklearn.feature_selection import VarianceThreshold
#p. 262